# Merge Player's Dataset and Match Dataset

In [24]:
import numpy as np
import pandas as pd
import csv
import os

In [33]:
df_ext = pd.read_excel('data/base_data.xlsx')

In [3]:
scraped_data = pd.read_csv('data/playersdata.csv', encoding='utf-8-sig')

In [4]:
# uniforming players' names and surnames 
newNames = []
for w in scraped_data['Player'] :
    surname = w.split()
    newName = surname[-1] + ' '
    for n in range(len(surname)-2, -1, -1):
        newName += surname[n][0] + '.'

    newNames.append(newName)

for i in scraped_data.index :
    scraped_data.at[i , 'Player'] = newNames[i]
    
    

In [5]:
def fill_values(row, scraped): 
    wElo = ''
    wBirthDate = ''
    wHand = ''
    wBackhand = ''
    wSurfaceElo = ''

    lElo = ''
    lBirthDate = ''
    lHand = ''
    lBackhand = ''
    lSurfaceElo = ''
        
    wFound = False
    lFound = False
    for i, r in scraped.iterrows() :
        if(row['Winner']  == r['Player']) :
            wFound = True
            wElo = r['Elo']
            wBirthDate = r['BirthDate']
            wHand = r['Hand']
            wBackhand = r['BackHand']
            if(row['Surface'] == 'Hard') :
                wSurfaceElo = r['hElo']
            if(row['Surface'] == 'Clay') :
                wSurfaceElo = r['cElo']
            if(row['Surface'] == 'Grass') :
                wSurfaceElo = r['gElo']
            
        elif (row['Loser']  == r['Player']) :
            lFound = True
            lElo = r['Elo']
            lBirthDate = r['BirthDate']
            lHand = r['Hand']
            lBackhand = r['BackHand']
            if(row['Surface'] == 'Hard') :
                lSurfaceElo = r['hElo']
            if(row['Surface'] == 'Clay') :
                lSurfaceElo = r['cElo']
            if(row['Surface'] == 'Grass') :
                lSurfaceElo = r['gElo']
        elif wFound and lFound :
            break
        
    print(row.name, row['Winner'], row['Loser'])
    return wElo, wSurfaceElo, wBirthDate, wHand, wBackhand, lElo, lSurfaceElo, lBirthDate, lHand, lBackhand

In [6]:
def join_datasets(base, scraped):
    result = base.apply(lambda row: fill_values(row, scraped), axis=1, result_type='expand')
            
    base = base.assign(WElo=result[0], WSurfElo=result[1], WBD=result[2], WHand=result[3], WBHand=result[4],
                  LElo=result[5], LSurfElo=result[6], LBD=result[7], LHand=result[8], LBHand=result[9])
    
    base['Date'] = pd.to_datetime(base['Date'])
    return base.sort_values(by='Date')
        

In [29]:
def write_dataset(base):
    os.chdir('data')
    with open('dataset.csv', 'w', newline='', encoding='utf-8-sig') as file :
        writer = csv.writer(file)
        writer.writerow([c for c in base.columns])
        for i, r in df_ext.iterrows():
            writer.writerow(r)
    os.chdir('..')
            
def split_by_year(base):
    os.chdir('data')
    for year in base.Date.dt.year.unique():
        base[base.Date.dt.year == year].to_csv(str(year) + '.csv', 
                                               encoding='utf-8-sig',
                                               index=False)
    os.chdir('..')

In [8]:
df_ext = join_datasets(df_ext, scraped_data)

220 Lopez F. Petzschner P.
221 Istomin D. De Bakker T.
475 Kendrick R. Gremelmayr D.
750 Bubka S. Gimeno-Traver D.
227 Roddick A. Matosevic M.
747 Tsonga J.W. Ramirez-Hidalgo R.
746 Garcia-Lopez G. Kukushkin M.
222 Berrer M. Sela D.
749 Marchenko I. Zverev M.
476 Phau B. Bolelli S.
224 Mayer F. Tomic B.
225 Giraldo S. Simon G.
226 Dolgopolov O. Andreev I.
480 Tipsarevic J. Schwank E.
479 Dodig I. Daniel M.
478 Robert S. Haider-Maurer A.
477 Nishikori K. Cilic M.
223 Kubot L. Brands D.
748 Davydenko N. Fognini F.
751 Chiudinelli M. El Amrani R.
753 Gabashvili T. Schuettler R.
754 Veic A. Sabry S.
755 Federer R. Schoorel T.
756 Nieminen J. Przysiezny M.
757 Lacko L. Riba P.
758 Kohlschreiber P. Seppi A.
759 Nadal R. Beck K.
760 Troicki V. Stakhovsky S.
761 Karlovic I. Rosol L.
752 Gulbis E. Hanescu V.
229 Becker B. Verdasco F.
228 Stepanek R. Kamke T.
489 Sugita Y. Brown D.
486 Falla A. Machado R.
487 Wawrinka S. Bopanna R.
488 Goffin D. Devvarman S.
230 Anderson K. Luczak P.
490 Kavcic 

3124 Stepanek R. Koubek S.
3127 Bogomolov A. Veic A.
2887 Bellucci T. Machado R.
2884 Zampieri C. Massu N.
2885 Fognini F. Ramos-Vinolas A.
2886 Gonzalez M. Souza J.
3135 Berrer M. Kavcic B.
3134 Cilic M. Kunitsyn I.
3133 Garcia-Lopez G. Lacko L.
3130 Marchenko I. Lajovic D.
3132 Mayer F. Stepanek R.
3131 Clement A. Kukushkin M.
2635 Soeda G. Schuettler R.
2639 Devvarman S. Huta Galung J.
2638 De Voest R. Przysiezny M.
2637 Dancevic F. Lopez F.
2636 Beck K. Dabul B.
2640 Van Der Merwe I. Brown D.
2641 Greul S. Raonic M.
2632 Huta Galung J. Tipsarevic J.
2633 Sela D. Roger-Vasselin E.
2634 Wolmarans F. Sijsling I.
2888 Robredo T. Mello R.
2890 Starace P. Andujar P.
3137 Gasquet R. Clement A.
3138 Ljubicic I. Brands D.
2891 Chela J.I. Zampieri C.
2889 Giraldo S. Andreev I.
3136 Bogomolov A. Petzschner P.
3139 Dodig I. Marchenko I.
2643 Anderson K. Sela D.
2644 Mannarino A. Gremelmayr D.
2645 Dancevic F. Wolmarans F.
2642 Beck K. Soeda G.
2892 Zeballos H. Nalbandian D.
2893 Giraldo S. Che

4989 Almagro N. Volandri F.
4991 Bellucci T. Navarro-Pastor I.
4992 Ferrer D. Gonzalez S.
4993 Almagro N. Giraldo S.
4994 Bellucci T. Kubot L.
5264 Fish M. Mello R.
5263 Tipsarevic J. Kavcic B.
5262 Nishikori K. Blake J.
4990 Monaco J. Cuevas P.
5266 Del Potro J.M. Gabashvili T.
5268 Nishikori K. Sweeting R.
5269 Tipsarevic J. Dodig I.
4995 Dolgopolov O. Wawrinka S.
5267 Anderson K. Serra F.
4996 Ferrer D. Monaco J.
4997 Almagro N. Bellucci T.
5531 Djokovic N. Berdych T.
5532 Federer R. Gasquet R.
5272 Tipsarevic J. Nishikori K.
5270 Fish M. Falla A.
5533 Djokovic N. Federer R.
5271 Del Potro J.M. Anderson K.
4998 Ferrer D. Dolgopolov O.
4999 Ferrer D. Almagro N.
5274 Del Potro J.M. Tipsarevic J.
5273 Del Potro J.M. Fish M.
6297 Hanescu V. Seppi A.
6296 Sweeting R. Granollers M.
6295 Young D. Starace P.
6294 Tipsarevic J. Kamke T.
6299 Berrer M. Phau B.
6300 Smyczek T. Marchenko I.
6301 De Voest R. Riba P.
6298 Devvarman S. Mannarino A.
6302 Schuettler R. Haase R.
6315 Petzschner P. Br

KeyboardInterrupt: 

In [9]:
write_dataset(df_ext)
split_by_year(df_ext)